In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.data import Field, BucketIterator

import numpy as np
from torchtext import data

import random
import math
import time

from transformers import T5Model,T5Tokenizer

In [2]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [3]:
tokenizer.vocab_size

32100

In [4]:
tokens = tokenizer.tokenize('Hello world how are you?')

print(tokens)

['▁Hello', '▁world', '▁how', '▁are', '▁you', '?']


In [5]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[8774, 296, 149, 33, 25, 58]


In [6]:
init_token = tokenizer.pad_token
eos_token = tokenizer.eos_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

<pad> </s> <pad> <unk>


In [7]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

0 1 0 2


In [8]:
max_input_length = tokenizer.max_model_input_sizes['t5-small']

print(max_input_length)

512


In [9]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [10]:
SRC = Field(batch_first = True,
          use_vocab = False,
          tokenize = tokenize_and_cut,
          preprocessing = tokenizer.convert_tokens_to_ids,
          init_token = init_token_idx,
          eos_token = eos_token_idx,
          pad_token = pad_token_idx,
          unk_token = unk_token_idx)

TRG = Field(batch_first = True,
          use_vocab = False,
          tokenize = tokenize_and_cut,
          preprocessing = tokenizer.convert_tokens_to_ids,
          init_token = init_token_idx,
          eos_token = eos_token_idx,
          pad_token = pad_token_idx,
          unk_token = unk_token_idx)

/home/gauranga/anaconda3/envs/pt/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [11]:
fields = [('src', SRC), ('trg', TRG)]

In [12]:
train_data = data.TabularDataset.splits(
                path = '',
                train = 'squad.csv',
                format = 'csv',
                fields = fields,
                skip_header = True)

train_data , valid_data = train_data[0].split(split_ratio=0.98,
                                             random_state = random.seed(4321))

/home/gauranga/anaconda3/envs/pt/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/home/gauranga/anaconda3/envs/pt/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [13]:
print(len(train_data.examples))
print(len(valid_data.examples))

57226
1168


In [14]:
print(vars(train_data.examples[25000]))

{'src': [2625, 3, 10, 16, 16433, 6, 3, 12687, 15, 157, 47, 2404, 13, 8, 3, 6856, 18, 9339, 3, 5379, 17, 6, 16, 84, 8, 3, 2160, 17, 1273, 10215, 1088, 4686, 12, 453, 8, 3, 2160, 17, 1273, 12568, 789, 16, 828, 5, 913, 12, 8, 648, 6, 3, 12687, 15, 157, 243, 6, 96, 13726, 80, 113, 65, 3, 12895, 3, 9, 508, 294, 13, 112, 280, 12, 8, 810, 13, 8, 892, 11, 8, 5559, 13, 5486, 6835, 500, 91, 24, 3, 9, 1088, 24, 5689, 3, 9, 23737, 789, 16, 579, 65, 1513, 66, 2233, 12, 8, 564, 3, 31, 10661, 138, 31, 5, 822, 3, 10, 125, 410, 3, 12687, 15, 157, 857, 47, 1513, 16, 8, 2925, 26518, 13, 3, 9, 23737, 789, 58], 'trg': [66, 2233, 12, 8, 564, 3, 31, 10661, 138, 31]}


In [15]:
src_tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[2000])['src'])
trg_tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[2000])['trg'])

print(src_tokens)
print(trg_tokens)

['▁context', '▁', ':', '▁in', '▁1957', ',', '▁bra', 'un', 'stein', '▁further', '▁demonstrated', '▁that', '▁the', '▁', 'r', 'udi', 'ment', 'ary', '▁devices', '▁could', '▁be', '▁used', '▁for', '▁non', '-', 'radi', 'o', '▁communication', '▁across', '▁', 'a', '▁short', '▁distance', '.', '▁question', '▁', ':', '▁what', '▁year', '▁was', '▁it', '▁discovered', '▁that', '▁early', '▁led', '▁instruments', '▁could', '▁be', '▁used', '▁for', '▁non', '-', 'radi', 'o', '▁communication', '?']
['▁1957']


In [16]:
device = torch.device('cuda')

BATCH_SIZE = 24

train_iterator, valid_iterator = BucketIterator.splits(
                                 (train_data, valid_data), 
                                 batch_size = BATCH_SIZE,
                                 device = device,
                                 sort_key=lambda x: len(x.src))

/home/gauranga/anaconda3/envs/pt/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [17]:
class T5Network(nn.Module):
    def __init__(self):
        
        super().__init__()
        
        self.t5 = T5Model.from_pretrained('t5-small')
        
        self.out = nn.Linear(self.t5.config.to_dict()['d_model'],
                             self.t5.config.to_dict()['vocab_size'])
                
    def forward(self, src, trg):
        
        embedded = self.t5(input_ids=src,decoder_input_ids=trg) 
        
        output = self.out(embedded[0])
        
        return output

In [18]:
model = T5Network().cuda()

Some weights of T5Model were not initialized from the model checkpoint at t5-small and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 76,988,544 trainable parameters


In [20]:
LEARNING_RATE = 0.0004

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [21]:
criterion = nn.CrossEntropyLoss(ignore_index = pad_token_idx)

In [22]:
N_EPOCHS = 4
CLIP = 1

best_valid_loss = float('inf')

In [23]:
for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    # TRAINING
    ##############################################################################
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(train_iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg[:,:-1])

        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)

        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    train_loss = epoch_loss / len(train_iterator)
    ##############################################################################
    
    # VALIDATION
    ##############################################################################
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(valid_iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg[:,:-1])
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    valid_loss = epoch_loss / len(valid_iterator)
    ##############################################################################
    
    end_time = time.time()
    
    print(f"EPOCH : {epoch+1}\tTRAIN LOSS : {train_loss:.2f}\tVALID LOSS : {valid_loss:.2f}\tTIME : {end_time-start_time:.2f}\n")

/home/gauranga/anaconda3/envs/pt/lib/python3.8/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


EPOCH : 1	TRAIN LOSS : 2.66	VALID LOSS : 0.90	TIME : 1027.51

EPOCH : 2	TRAIN LOSS : 0.67	VALID LOSS : 0.60	TIME : 1042.50

EPOCH : 3	TRAIN LOSS : 0.38	VALID LOSS : 0.53	TIME : 1045.49

EPOCH : 4	TRAIN LOSS : 0.27	VALID LOSS : 0.53	TIME : 1050.51



In [24]:
# CONVERT ALL MODEL WEIGHTS AND BIASES TO HALF PRECISION
# MODEL SIZE WILL REDUCE
model = model.half()

In [25]:
torch.save(model.state_dict(), 't5_qa_model.pt')

In [26]:
def translate_sentence2(sentence, eval_model, device, max_len = 50):
    
    eval_model.eval()

    src_indexes = [init_token_idx] + sentence + [eos_token_idx]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    trg_indexes = [init_token_idx]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
        
        with torch.no_grad():
            output = eval_model(src_tensor, trg_tensor)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == eos_token_idx:
            break

    return trg_indexes[1:]

In [27]:
idxs = random.sample(range(0, len(valid_data.examples)), 20)
for i in idxs:
    src = vars(valid_data.examples[i])['src']
    trg = vars(valid_data.examples[i])['trg']
    translation = translate_sentence2(src, model, device)

    print(f"SRC : {''.join(tokenizer.convert_ids_to_tokens(src))}")
    print(f"TRG : {''.join(tokenizer.convert_ids_to_tokens(trg))}")
    print(f"PRED : {''.join(tokenizer.convert_ids_to_tokens(translation))}\n")

SRC : ▁context▁:▁in▁1979,▁the▁japanese▁state▁broadcaster▁nhk▁first▁developed▁consumer▁high-definition▁television▁with▁a▁5:3▁display▁aspect▁ratio.▁question▁:▁in▁what▁year▁did▁nhk▁first▁develop▁consumer▁hd▁television▁with▁a▁5:3▁aspect▁ratio?
TRG : ▁1979
PRED : ▁1979</s>

SRC : ▁context▁:▁the▁company▁aims▁to▁reduce▁its▁external▁environmental▁impact▁through▁energy-efficient▁evolution▁of▁products,▁and▁also▁reduce▁its▁direct▁operational▁impact▁through▁energy-efficiency▁programs.▁internal▁energy-efficiency▁programs▁reportedly▁save▁the▁company▁more▁than▁$3▁million▁annually▁in▁energy-cost▁savings.▁the▁largest▁component▁of▁the▁company's▁internal▁energy-efficiency▁savings▁comes▁through▁pc▁power▁management:▁the▁company▁expects▁to▁save▁$1.8▁million▁in▁energy▁costs▁through▁using▁specialized▁energy-management▁software▁on▁a▁network▁of▁50,000▁pcs.▁question▁:▁what▁does▁dell▁aim▁to▁reduce▁by▁creating▁energy▁efficient▁products?
TRG : ▁external▁environmental▁impact
PRED : ▁external▁environmental▁impact</s>